# 使用selenium和bs4爬虫

## 1.selenium的使用

In [ ]:
import os,time
from selenium import webdriver
from bs4 import BeautifulSoup
chromedriver = "C:\\Program Files (x86)\\Google\\Chrome\\Application\\chromedriver.exe"#chrome所在驱动
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get('http://user.100xuexi.com/MyDeal/myEBookPurchase.aspx')
time.sleep(2)

In [ ]:
driver.find_element_by_id('username').send_keys('18775325832')#

In [ ]:
driver.find_element_by_id('txtVerify').clear()#验证码
driver.find_element_by_id('txtVerify').send_keys('69')#验证码

In [ ]:
#driver.find_element_by_xpath('//*[@id="conPurchaseA1"]/table/tbody/tr[3]/td[2]/a').click();
driver.find_element_by_xpath('//*[@id="regNow"]').click();

In [ ]:
driver.find_element_by_xpath('//*[@id="PurchaseA2"]').click();

In [ ]:
#接下来会有新的窗口打开，获取所有窗口句柄
windows = driver.window_handles
driver.switch_to.window(windows[-1])

In [ ]:
driver.find_element_by_xpath('//*[@id="lefter"]/div/div[1]/a').click();
#/PracticeCenter/Chapter/Index?TQuestionPlanID=2997&groupName=&code=&tb_l_PaperQuePlanID=86215&Model=chapter&TypeMenuFlag=2
#/PracticeCenter/Chapter/Index?TQuestionPlanID=2997&groupName=&code=&tb_l_PaperQuePlanID=86216&Model=chapter&TypeMenuFlag=2
#item表示章，li后的数字表示节

In [ ]:
#driver.find_element_by_xpath('//*[@id="item01"]/li[2]/a').click();

## 2.解析成html

In [7477]:
html = driver.page_source
soup = BeautifulSoup(html, "html5lib")

## 3.html文件解析

### 1.解析问题

In [7478]:
import re

In [7479]:
baseurl_item = u'2Z208000　解决建设工程纠纷法律制度\\2Z208050　行政强制、行政复议和行政诉讼制度'
single_baseurl = u'二级建造师\\法规\\'+ baseurl_item + u'\\单选\\'
multiple_baseurl = u'二级建造师\\法规\\'+ baseurl_item + u'\\多选\\'

In [7480]:
content_all = soup.find_all("div",attrs={"class":"nodeItem"})

In [7481]:
#soup.find('div',class_='ItemBotCompose').decompose()

In [7482]:
# replace with `soup.findAll` if you are using BeautifulSoup3
for div in soup.find_all("div", {'class':'Item config'}): 
    if len(div.find_all("div", {'class':'ItemBotCompose'}))!=0:
        div.decompose() #将满足调节的 div 项删除

In [7483]:
single_content_all = []
multiple_content_all = []

single_content_all = option_all[0].find_all("div",attrs={"class":"question config"})
if len(content_all)>1:
    multiple_content_all = content_all[1].find_all("div",attrs={"class":"question config"})

In [7484]:
len(single_content_all)

21

单选

In [7485]:
single_question = []

for i in range(len(single_content_all)):
    single_contents = single_content_all[i].dt.contents
    #print(contents_single)
    single_s = ''
    for j in single_contents:#单选
        single_s = single_s+str(j)
    single_question.append(re.sub('[\n]','',single_s))

In [7486]:
print(len(single_question))

21


多选

In [7487]:
multiple_question = []

for i in range(len(multiple_content_all)):#多选
    multiple_contents = multiple_content_all[i].dt.contents
    #print(contents_single)
    multiple_s = ''
    for j in multiple_contents:
        multiple_s = multiple_s+str(j)
    multiple_question.append(re.sub('[\n]','',multiple_s))

In [7488]:
print(len(multiple_question))

13


保存数据

In [7489]:
#question_file = open('question_file.txt', 'w')
import codecs

single_folder = os.path.exists(single_baseurl)
multiple_folder = os.path.exists(multiple_baseurl)
if not single_folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
    os.makedirs(single_baseurl)
                
if not multiple_folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
    os.makedirs(multiple_baseurl)
                
single_question_file = codecs.open(single_baseurl + 'single_question.txt', 'w','utf-8')
multiple_question_file = codecs.open(multiple_baseurl + 'multiple_question.txt', 'w','utf-8')

for item in single_question:
    single_question_file.write("%s\r\n" % item)
    
for item in multiple_question:
    multiple_question_file.write("%s\r\n" % item)
    
single_question_file.close()
multiple_question_file.close()

### 2.解析选项

In [7490]:
option_all = soup.find_all("div",attrs={"class":"nodeItem"})

区别单选和多选

In [7491]:
single_option_all = option_all[0].find_all("div",attrs={"class":"question config"})
if len(option_all)>1:
    multiple_option_all = option_all[1].find_all("div",attrs={"class":"question config"})

In [7492]:
len(single_option_all)

21

In [7493]:
len(b)

1

In [7494]:
len(multiple_option_all)

13

单选

In [7495]:
single_option_a = []
single_option_b = []
single_option_c = []
single_option_d = []
for option_i in range(len(single_option_all)):
    single_option_a.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[0].get_text()))
    single_option_b.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[1].get_text()))
    single_option_c.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[2].get_text()))
    single_option_d.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[3].get_text()))

多选

In [7496]:
multiple_option_a = []
multiple_option_b = []
multiple_option_c = []
multiple_option_d = []
multiple_option_e = []
for option_i in range(len(multiple_option_all)):
    multiple_option_a.append(re.sub('[\r|\n|\t|\\s]', '', multiple_option_all[option_i].table.tbody.find_all("label")[0].get_text()))
    multiple_option_b.append(re.sub('[\r|\n|\t|\\s]', '', multiple_option_all[option_i].table.tbody.find_all("label")[1].get_text()))
    multiple_option_c.append(re.sub('[\r|\n|\t|\\s]', '', multiple_option_all[option_i].table.tbody.find_all("label")[2].get_text()))
    multiple_option_d.append(re.sub('[\r|\n|\t|\\s]', '', multiple_option_all[option_i].table.tbody.find_all("label")[3].get_text()))
    multiple_option_e.append(re.sub('[\r|\n|\t|\\s]', '', multiple_option_all[option_i].table.tbody.find_all("label")[4].get_text()))

In [7497]:
single_option_a[1]

'A．建设单位的违约行为'

保存单选选项

In [7498]:
import codecs

single_option_a_file = codecs.open(single_baseurl + 'single_option_a.txt', 'w','utf-8')
single_option_b_file = codecs.open(single_baseurl + 'single_option_b.txt', 'w','utf-8')
single_option_c_file = codecs.open(single_baseurl + 'single_option_c.txt', 'w','utf-8')
single_option_d_file = codecs.open(single_baseurl + 'single_option_d.txt', 'w','utf-8')

for item in single_option_a:
    single_option_a_file.write("%s\r\n" % item)
for item in single_option_b:
    single_option_b_file.write("%s\r\n" % item)
    
for item in single_option_c:
    single_option_c_file.write("%s\r\n" % item)
    
for item in single_option_d:
    single_option_d_file.write("%s\r\n" % item)

single_option_a_file.close()
single_option_b_file.close()
single_option_c_file.close()
single_option_d_file.close()

保存多选选项

In [7499]:
import codecs

multiple_option_a_file = codecs.open(multiple_baseurl + 'multiple_option_a.txt', 'w','utf-8')
multiple_option_b_file = codecs.open(multiple_baseurl + 'multiple_option_b.txt', 'w','utf-8')
multiple_option_c_file = codecs.open(multiple_baseurl + 'multiple_option_c.txt', 'w','utf-8')
multiple_option_d_file = codecs.open(multiple_baseurl + 'multiple_option_d.txt', 'w','utf-8')

for item in multiple_option_a:
    multiple_option_a_file.write("%s\r\n" % item)
for item in multiple_option_b:
    multiple_option_b_file.write("%s\r\n" % item)
    
for item in multiple_option_c:
    multiple_option_c_file.write("%s\r\n" % item)
    
for item in multiple_option_d:
    multiple_option_d_file.write("%s\r\n" % item)

multiple_option_a_file.close()
multiple_option_b_file.close()
multiple_option_c_file.close()
multiple_option_d_file.close()

### 解析答案

In [7500]:
anwser_all = soup.find_all("div",attrs={"class":"nodeItem"})

In [7501]:
single_anwser_all = anwser_all[0].find_all("div",attrs={"class":"Item config"}) #单选
if len(anwser_all)>1:
    multiple_anwser_all = anwser_all[1].find_all("div",attrs={"class":"Item config"}) #多选

In [7502]:
len(single_anwser_all)

21

In [7503]:
single_answer_right = []
multiple_answer_right = []

for answer_i in range(len(single_anwser_all)):
    #print(anwser_all[25].find("div",attrs={"class":"config answer"}))
    #print(anwser_all[answer_i].find("table",attrs={"class":"HiddenStart"}).find("div",attrs={"class":"anser Con"}))
    #print(anwser_all[answer_i].find("table",attrs={"class":"HiddenStart"}).find("div",attrs={"class":"anser Con"}).get_text())
    single_answer_right.append(single_anwser_all[answer_i].find("div",attrs={"class":"anser Con"}).string)
    
for answer_i in range(len(multiple_anwser_all)):
    #print(anwser_all[25].find("div",attrs={"class":"config answer"}))
    #print(anwser_all[answer_i].find("table",attrs={"class":"HiddenStart"}).find("div",attrs={"class":"anser Con"}))
    #print(anwser_all[answer_i].find("table",attrs={"class":"HiddenStart"}).find("div",attrs={"class":"anser Con"}).get_text())
    multiple_answer_right.append(multiple_anwser_all[answer_i].find("div",attrs={"class":"anser Con"}).string)
    
len(single_answer_right)

21

In [7504]:
import codecs

single_anwser_file = codecs.open(single_baseurl + 'single_anwser.txt', 'w','utf-8')
multiple_anwser_file = codecs.open(multiple_baseurl + 'multiple_anwser.txt', 'w','utf-8')

for item in multiple_answer_right:
    multiple_anwser_file.write("%s\r\n" % item)
    
single_anwser_file.close()
multiple_anwser_file.close()

In [7505]:
#print(re.sub('[\r|\n|\t|\\s]', '', anwser_1))

### 解析解析

In [7506]:
analysis_all = soup.find_all("div",attrs={"class":"nodeItem"})

In [7507]:
single_analysis_all = analysis_all[0].find_all("div",attrs={"class":"Item config"}) #单选
if len(analysis_all)>1:
    multiple_analysis_all = analysis_all[1].find_all("div",attrs={"class":"Item config"}) #多选
len(single_analysis_all)

21

In [7508]:
single_analysis = []
multiple_analysis = []

for analysis_i in range(len(single_analysis_all)):
    print(single_analysis_all[analysis_i].find("div",attrs={"class":"analysits Con"}))
    print('\n')
    #answer_right.append(anwser_all[answer_i].find("div",attrs={"class":"analysits Con"}).string)
#print(answer_right)
for analysis_i in range(len(multiple_analysis_all)):
    print(multiple_analysis_all[analysis_i].find("div",attrs={"class":"analysits Con"}))
    print('\n')

<div class="analysits Con">AC两项，行政机关依法做出要求当事人履行排除妨碍、恢复原状等义务的行政决定，当事人逾期不履行，经催告仍不履行，其后果已经或者将危害交通安全、造成环境污染或者破坏自然资源的，行政机关可以代履行，或者委托没有利害关系的第三人代履行。D项，催告当事人履行，当事人履行的，停止代履行。</div>


<div class="analysits Con">行政复议的目的，是为了防止和纠正违法的或者不当的具体行政行为，保护公民、法人和其他组织的合法权益，保障和监督行政机关依法行使职权。因此，只要是公民、法人或者其他组织认为行政机关的具体行政行为侵犯其合法权益，就有权向行政机关提出行政复议申请。</div>


<div class="analysits Con">根据《行政复议法》规定，有11项可申请行政复议的情形，结合建设工程实践，其中7种尤为重要：①对行政机关作出的警告、罚款、没收违法所得、没收非法财物、责令停产停业、暂扣或者吊销许可证、暂扣或者吊销执照、行政拘留等行政处罚决定不服的；②对行政机关作出的限制人身自由或者查封、扣押、冻结财产等行政强制措施决定不服的；③对行政机关作出的有关许可证、执照、资质证、资格证等证书变更、中止、撤销的决定不服的；④认为行政机关侵犯合法的经营自主权的；⑤认为行政机关违法集资、征收财物、摊派费用或者违法要求履行其他义务的；⑥认为符合法定条件，申请行政机关颁发许可证、执照、资质证、资格证等证书，或者申请行政机关审批、登记有关事项，行政机关没有依法办理的；⑦认为行政机关的其他具体行政行为侵犯其合法权益的。根据《行政复议法》第八条的规定，不服行政机关作出的行政处分或者其他人事处理决定的，应当依照有关法律、行政法规的规定提起申诉，而不能提起行政复议。</div>


<div class="analysits Con">行政复议是指行政机关根据上级行政机关对下级行政机关的监督权，在当事人的申请和参加下，按照行政复议程序对具体行政行为进行合法性和适当性审查，并作出决定以解决行政侵权争议的活动。不服行政机关作出的行政处分或者其他人事处理决定的，应当依照有关法律、行政法规的规定提起申诉，不能提起行政复议。</div>


<div class="analysits Con">对于行政复议，应当按照《行政

In [7509]:
print(multiple_analysis_all[0].find("div",attrs={"class":"analysits Con"}))

<div class="analysits Con">下列事项应按规定的纠纷处理方式解决，不能提起行政复议：①不服行政机关作出的行政处分或者其他人事处理决定的，应当依照有关法律、行政法规的规定提起申诉；②不服行政机关对民事纠纷作出的调解或者其他处理，应当依法申请仲裁或者向法院提起诉讼。</div>


In [7510]:
single_analay = []
multiple_analay = []
for i in range(len(single_analysis_all)):
    single_analay_item = single_analysis_all[i].find("div",attrs={"class":"analysits Con"}).contents
    #print(analay_single)
    single_analay.append(single_analay_item)
    #print(analay_single)
    
for i in range(len(multiple_analysis_all)):
    multiple_analay_item = multiple_analysis_all[i].find("div",attrs={"class":"analysits Con"}).contents
    #print(analay_single)
    multiple_analay.append(multiple_analay_item)
    #print(analay_single)

In [7511]:
single_analay_final = []
for item in single_analay:
    #analysis_file.write("%s" % item)
    single_s = ''
    for i in item:
        single_s = single_s + str(i)
    single_analay_final.append(single_s)
    #analysis_file.write("%s\n" % s)
#analysis_file.close()

multiple_analay_final = []
for item in multiple_analay:
    #analysis_file.write("%s" % item)
    multiple_s = ''
    for i in item:
        multiple_s = multiple_s + str(i)
    multiple_analay_final.append(multiple_s)
    #analysis_file.write("%s\n" % s)
#analysis_file.close()

In [7512]:
import codecs
single_analysis_file = codecs.open(single_baseurl + 'single_analysis_file.txt', 'w','utf-8')

for s in single_analay_final:
    print(s)
    print("\n")
    single_analysis_file.write("%s\r\n" % s)
    
multiple_analysis_file = codecs.open(multiple_baseurl + 'multiple_analysis_file.txt', 'w','utf-8')

for s in multiple_analay_final:
    print(s)
    print("\n")
    multiple_analysis_file.write("%s\r\n" % s)    
    
single_analysis_file.close()
multiple_analysis_file.close()

AC两项，行政机关依法做出要求当事人履行排除妨碍、恢复原状等义务的行政决定，当事人逾期不履行，经催告仍不履行，其后果已经或者将危害交通安全、造成环境污染或者破坏自然资源的，行政机关可以代履行，或者委托没有利害关系的第三人代履行。D项，催告当事人履行，当事人履行的，停止代履行。


行政复议的目的，是为了防止和纠正违法的或者不当的具体行政行为，保护公民、法人和其他组织的合法权益，保障和监督行政机关依法行使职权。因此，只要是公民、法人或者其他组织认为行政机关的具体行政行为侵犯其合法权益，就有权向行政机关提出行政复议申请。


根据《行政复议法》规定，有11项可申请行政复议的情形，结合建设工程实践，其中7种尤为重要：①对行政机关作出的警告、罚款、没收违法所得、没收非法财物、责令停产停业、暂扣或者吊销许可证、暂扣或者吊销执照、行政拘留等行政处罚决定不服的；②对行政机关作出的限制人身自由或者查封、扣押、冻结财产等行政强制措施决定不服的；③对行政机关作出的有关许可证、执照、资质证、资格证等证书变更、中止、撤销的决定不服的；④认为行政机关侵犯合法的经营自主权的；⑤认为行政机关违法集资、征收财物、摊派费用或者违法要求履行其他义务的；⑥认为符合法定条件，申请行政机关颁发许可证、执照、资质证、资格证等证书，或者申请行政机关审批、登记有关事项，行政机关没有依法办理的；⑦认为行政机关的其他具体行政行为侵犯其合法权益的。根据《行政复议法》第八条的规定，不服行政机关作出的行政处分或者其他人事处理决定的，应当依照有关法律、行政法规的规定提起申诉，而不能提起行政复议。


行政复议是指行政机关根据上级行政机关对下级行政机关的监督权，在当事人的申请和参加下，按照行政复议程序对具体行政行为进行合法性和适当性审查，并作出决定以解决行政侵权争议的活动。不服行政机关作出的行政处分或者其他人事处理决定的，应当依照有关法律、行政法规的规定提起申诉，不能提起行政复议。


对于行政复议，应当按照《行政复议法》的规定向有权受理的行政机关申请，如对县级以上地方各级人民政府工作部门的具体行政行为不服的，由申请人选择，可以向该部门的本级人民政府申请行政复议，也可以向上一级主管部门申请行政复议。


《行政诉讼法》第十三条规定，人民法院不受理公民、法人或者其他组织对下列事项提起的诉讼：①国防、外交等国家行为；②行政法规、规

单选选项拼接

In [7513]:
"""
question
option_a
option_b
option_c
option_d
answer_right
analay_final
"""

"""
option= codecs.open(baseurl + 'option.txt', 'w','utf-8')

for i in range(len(option_a)):
    option.write(option_a[i].strip().replace(u'A\uff0e','')
                 +u'<br>'+option_b[i].strip().replace(u'B\uff0e','')
                 +u'<br>'+option_c[i].strip().replace(u'C\uff0e','')
                 +u'<br>'+option_d[i].replace(u'D\uff0e','')
                 )
"""
single_data = []
for i in range(len(single_option_a)):
    single_data_string = single_question[i].replace('（　　）','{{c1::（）}}') + "	"+ single_option_a[i].strip().replace('A．','')+ '<br>' + single_option_b[i].strip().replace('B．','')+ '<br>'+ single_option_c[i].strip().replace('C．','')+ '<br>'+ single_option_d[i].replace('D．','')+ "	"+ single_answer_right[i]+ "	"+ single_analay_final[i]
    single_data.append(single_data_string)
data_single = codecs.open(single_baseurl + 'data_single.txt', 'w','utf-8')

for i in range(len(single_data)):
    data_single.write(single_data[i]+"\r\n")
data_single.close()

In [7514]:
len(single_data )

21

多选选项拼接

In [7515]:
multiple_data = []
for i in range(len(multiple_option_a)):
    multiple_data_string = multiple_question[i].replace('（　　）','{{c1::（）}}') + "	"+ multiple_option_a[i].strip().replace('A．','')+ '<br>' + multiple_option_b[i].strip().replace('B．','')+ '<br>'+ multiple_option_c[i].strip().replace('C．','')+ '<br>'+ multiple_option_d[i].replace('D．','')+ '<br>'+ multiple_option_e[i].replace('E．','')+ "	"+ multiple_answer_right[i]+ "	"+ multiple_analay_final[i]
    multiple_data.append(multiple_data_string)
data_multiple = codecs.open(multiple_baseurl + 'data_multiple.txt', 'w','utf-8')

for i in range(len(multiple_data)):
    data_multiple.write(multiple_data[i]+"\r\n")
data_multiple.close()

In [7516]:
len(multiple_data)

13